Import Packages

In [1]:
import pandas as pd
import networkx as nx
from datetime import datetime, timedelta

Import file

In [2]:
# Create a DataFrame
df = pd.read_csv('4f-lv-final-export.csv', sep=';', encoding='ISO-8859-1')

# Convert 'time' column to datetime
df['time'] = pd.to_datetime(df['time'])

Filter unnecessary columns

In [3]:
# columns = ["token", "fingerprint", "userToken", "siteId", "userId", "url", "type", "group",
#            "category", "action", "name", "time", "visitKey", "groupSource"]
# columns_to_keep = ["token", "userToken", "userId", "url", "type", "category", "time"]
# may_be = ["visitKey", "groupSource"]

# TODO: 1. Find start and end point of time
# TODO: 2. Create id table for url. Use "page_id" (newly made one), "url", "category" as "page_category" STRING, "type" as "page_type" STRING
# TODO: 3. Create node table with "page_id" as INT64, visits_last_7_days, unique_visitors_last_7_days, trend_percentage
# TODO: 4. Create edge table with "source_page_id" as INT64, "destination_page_id" as INT64, "transition_count" as INT64,
# TODO    "avg_time_between_pages" as FLOAT64, "last_visit_timestamp TIMESTAMP", "avg_path_depth as FLOAT64"

# Filter columns with necessary information only
def sort_and_filter_clicksream_df(df):
    df_sorted = df.sort_values(by=['token', 'time'])
    columns_to_keep = ["token", "userToken", "userId", "url", "type", "category", "time", "visitKey", "groupSource"]
    return df_sorted[columns_to_keep]

Create URL table (url_table) for storing page (node) information for each page_id
Also create merge_url function for future

In [4]:
def create_url_table(df):
    # Select the relevant columns and drop duplicates to get unique URLs
    url_table = df[['url', 'category', 'type']].drop_duplicates()
    
    # Add a unique page_id for each unique URL
    url_table = url_table.reset_index(drop=True)
    url_table['page_id'] = url_table.index + 1  # Start page_id from 1
    
    # Rename columns as specified
    url_table = url_table.rename(columns={
        'category': 'page_category',
        'type': 'page_type'
    })
    
    # Reorder columns to match the required output
    url_table = url_table[['page_id', 'url', 'page_category', 'page_type']]
    
    return url_table

def replace_url_with_id(df, url_table):
    # Merge df_filtered with url_table on 'url' to get the corresponding 'page_id'
    df_merged = df.merge(url_table[['page_id', 'url']], on='url', how='left')
    # Drop the original 'url' column and rename 'page_id' column
    df_merged = df_merged.drop(columns=['url']).rename(columns={'page_id': 'page_id'})
    return df_merged

def merge_url(df, url_table):
    df_merged = df.merge(url_table, on='page_id', how='left')
    return df_merged

In [5]:
def create_edges_table(df):
    # Sort by token and time for each user's navigation sequence
    df_sorted = df.sort_values(by=['token', 'time'])
    
    # Shift data to get source and destination pages and times
    df_sorted['next_page_id'] = df_sorted.groupby('token')['page_id'].shift(-1)
    df_sorted['source_time'] = df_sorted['time']
    df_sorted['destination_time'] = df_sorted.groupby('token')['time'].shift(-1)
    
    # Remove rows where next_page_id or destination_time is NaN (last page in each token sequence)
    edges = df_sorted.dropna(subset=['next_page_id', 'destination_time'])
    
    # Rename and select relevant columns for edges
    edges = edges.rename(columns={'page_id': 'source_page_id', 'next_page_id': 'destination_page_id'})
    
    # Add depth, which is the order of the visit for each token group
    edges['depth'] = edges.groupby('token').cumcount() + 1
    
    # Calc time difference
    edges['time_diff'] = (edges['destination_time'] - edges['source_time']).dt.total_seconds()
    
    # Ensure ids are in int
    edges['source_page_id'] = edges['source_page_id'].astype(int)
    edges['destination_page_id'] = edges['destination_page_id'].astype(int)
    
    # Select columns for the edge data
    edge_data = edges[['token', 'source_page_id', 'destination_page_id', 'source_time', 'destination_time', 'time_diff', 'depth']]
    
    return edge_data


Create a function that export edges from set time frame then aggregate edges and calculate averages


In [6]:
def create_edges_table_export(edges_df, time, duration):
    """
    Generate an aggregated export edges table from whole edges table based on unique edges within a specified time range.
    Parameters:
        edges_df (DataFrame): The DataFrame containing edge data.
        time (str): The start date for filtering in 'YYYY-MM-DD' format.
        duration (int): The number of days from the start time to determine the end time.
    Returns:
        DataFrame: Aggregated edges table with the required columns.
    """
    time = pd.to_datetime(time)
    if duration >= 0: # Determine the start and end of the time range based on duration
        start_time = time
        end_time = time + timedelta(days=duration)
    else:
        start_time = time + timedelta(days=duration)
        end_time = time
    
    # Filter edges within the specified time range
    edges_filtered = edges_df[(edges_df['source_time'] >= start_time) & (edges_df['source_time'] <= end_time)]
    
    # Aggregate data by unique edges (source_page_id, destination_page_id)
    edges_table_export = edges_filtered.groupby(['source_page_id', 'destination_page_id']).agg(
        transition_count=('depth', 'count'),
        avg_time_between_pages=('time_diff', 'mean'),
        last_visit_timestamp=('destination_time', 'max'),
        avg_path_depth=('depth', 'mean'),
        unique_token_count=('token', 'nunique')
    ).reset_index()
    
    # Ensure correct data types for each columns
    edges_table_export['source_page_id'] = edges_table_export['source_page_id'].astype('int64')
    edges_table_export['destination_page_id'] = edges_table_export['destination_page_id'].astype('int64')
    edges_table_export['transition_count'] = edges_table_export['transition_count'].astype('int64')
    edges_table_export['avg_time_between_pages'] = edges_table_export['avg_time_between_pages'].astype('float64')
    edges_table_export['avg_path_depth'] = edges_table_export['avg_path_depth'].astype('float64')
    edges_table_export['unique_token_count'] = edges_table_export['unique_token_count'].astype('int64')

    return edges_table_export

def create_nodes_table_export(edges_df, time, n_days):
    # TODO: may need to change outcome of the trend result. Note that it is replacing value with 99999 if 0 is detected for prev result
    """
    Generate the nodes table export based on edge data.
    Parameters:
        edges_df (DataFrame): The DataFrame containing edge data.
        time (str): The reference date for filtering in 'YYYY-MM-DD' format.
        n (int): Number of days for the last period and the previous period comparison.
    Returns:
        DataFrame: Nodes table with the required columns.
    """
    time = pd.to_datetime(time)
    last_period_start = time - timedelta(days=n_days)
    prev_period_start = time - timedelta(days=2 * n_days)
    
    # Get edges within the last n days and previous n days
    last_period_edges = create_edges_table_export(edges_df, last_period_start, n_days)
    prev_period_edges = create_edges_table_export(edges_df, prev_period_start, n_days)
    
    # Aggregate visits and unique visitors (sum of unique_token_count) in the last n days
    last_period_visits = last_period_edges.groupby('destination_page_id').agg(
        visits_last_n_days=('transition_count', 'sum'),
        unique_visitors_last_n_days=('unique_token_count', 'sum')  # Sum unique token counts for each destination page
    ).reset_index()
    
    # Aggregate visits in the previous n days
    prev_period_visits = prev_period_edges.groupby('destination_page_id').agg(
        prev_visits=('transition_count', 'sum')
    ).reset_index()
    
    # Merge data for trend calculation
    nodes_table_export = pd.merge(last_period_visits, prev_period_visits, on='destination_page_id', how='left').fillna(0)
    
    # Calculate trend percentage with the condition for 99999 when prev_visits is 0
    nodes_table_export['trend_percentage'] = nodes_table_export.apply(
        lambda row: 99999 if row['prev_visits'] == 0 else
        ((row['visits_last_n_days'] - row['prev_visits']) / row['prev_visits']) * 100, axis=1
    )
    
    # Rename destination_page_id to page_id for consistency
    nodes_table_export = nodes_table_export.rename(columns={'destination_page_id': 'page_id'})
    
    # Ensure correct data types
    nodes_table_export['page_id'] = nodes_table_export['page_id'].astype('int64')
    nodes_table_export['visits_last_' + str(n_days) + '_days'] = nodes_table_export['visits_last_n_days'].astype('int64')
    nodes_table_export['unique_visitors_last_' + str(n_days) + '_days'] = nodes_table_export['unique_visitors_last_n_days'].astype('int64')
    nodes_table_export['trend_percentage'] = nodes_table_export['trend_percentage'].astype('float64')
    
    nodes_table_export = nodes_table_export.drop(columns=['visits_last_n_days', 'unique_visitors_last_n_days'])

    return nodes_table_export


def export_edges_and_nodes(edges_df, url_table, time, days = 7):
    edges_export = create_edges_table_export(edges_df, time, days)
    nodes_export = create_nodes_table_export(edges_df, time, days)
    
    # Drop unnecessary tables
    edges_export = edges_export.drop(columns=['unique_token_count'])
    nodes_export = nodes_export.drop(columns=['prev_visits'])
    
    nodes_export_with_url = merge_url(nodes_export, url_table)
    
    return edges_export, nodes_export, url_table, nodes_export_with_url

Check functions

In [7]:
df_filtered = sort_and_filter_clicksream_df(df)
url_table = create_url_table(df_filtered)
# (optional) check times
# start_time = df_filtered['time'].min()
# end_time = df_filtered['time'].max()
# print("Start Time:", start_time)
# print("End Time:", end_time)
df_filtered = replace_url_with_id(df_filtered, url_table)
edge_data = create_edges_table(df_filtered)
edges_export, nodes_export, url_table, nodes_export_with_url = export_edges_and_nodes(edge_data, url_table, "2023-04-15", days = 7)

In [8]:
print(edges_export)

        source_page_id  destination_page_id  transition_count  \
0                    2                 3349               159   
1                    3                  757               366   
2                    7                    7                12   
3                    7                   35                18   
4                    7                   36                 1   
...                ...                  ...               ...   
111695          488275                  276                 1   
111696          488275                  999                 1   
111697          488275                 1349                 1   
111698          488275                 7055                11   
111699          488275                55156                 1   

        avg_time_between_pages last_visit_timestamp  avg_path_depth  
0                     0.000000  2023-04-21 06:23:05       13.220126  
1                     0.000000  2023-04-21 21:13:56       35.103825  
2        

In [9]:
print(nodes_export_with_url)

       page_id  trend_percentage  visits_last_7_days  \
0            2        -46.969697                 105   
1            3        -75.484872                1264   
2            7         19.354839                 111   
3            8        -77.813318                 763   
4           10        -97.981497                  24   
...        ...               ...                 ...   
42498   488217        -33.333333                   6   
42499   488223      99999.000000                  21   
42500   488235        480.000000                  29   
42501   488240        -33.333333                   2   
42502   488275        -25.000000                   9   

       unique_visitors_last_7_days  \
0                               46   
1                              751   
2                              110   
3                              409   
4                               18   
...                            ...   
42498                            1   
42499                  

In [10]:
edge_data.to_csv('edge_data.csv', index=False)
edges_export.to_csv('edges_export.csv', index=False)
nodes_export.to_csv('nodes_export.csv', index=False)
url_table.to_csv('url_table.csv', index=False)
nodes_export_with_url.to_csv('nodes_export_with_url.csv', index=False)